## Versão 1.1 - Magnum
### Estado
1. Indicador Ecônimo - IPCA
2. Valor do Portfólio  -> Influênciado pelas Ações
3. Atual Alocação -> Influênciado pelas Ações
4. Preços dos Ativos disponíveis para compra
### Ações
* Alocar Valor do Portfólio entre os ativos
### Política
* Estocástica -> Ação = Sample de Distribuição gerada pela politica:
1.  Via Rede Neural -> Rede Gera 2 Parametros que serão usados para gerar uma distribuição gaussiana
* Estamos usando e-greedy para controlar exploitation e exploration
### Episódio
* 30 dias de Alocação
### Step
* 1 dia de Alocar e verificar o Resultado em relação ao dia seguinte
* Reward: Lucro
### Treinamento
* Percorre 30 dias calculando Recompensa Acumulada Ajustada
* Calcula o Gradiente de Política
* Ajusta a Rede Neural -> parametros da rede = parametros da rede + lr *  gradiente de politica
### Para Averiguar
* Verificar cubo de gradiente de alocação
* Forjar série temporal que saibamos o comportamento que queremos para testar o aprendizado do modelo:
    * Linear com ruído pra cima e pra baixo
    * Variando Pra caceta -> Seno
* Caso Extremo:
    * Learning Rate e Desvio Padrão Altos -> Altissima exploração e nada de aprender
    * Learning Rate e Desvio Padrão Baixissimo -> Não explora nada
* Buscar Convergência fora dos vértices
* Possíveis Dados Extras:
    * Indicadores Econômicos:
        * Features -> Média Móvel
        * Risco -> Sharpe ratio, Value at Risk

* Learning Rate -> Verificar Mudanças
* Para Depois:
    * Funções de Recompensa
    * Evoluções de Rede Neural
* Qual o Ideal:
    * 
* Usar ou não usar política de exploração estilo e-greedy ou ruído faz sentido? (bernardo acredita que a maneira de modelar isso seria via desvio padrão - mas não sei como).



In [1]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn
import os
import mlflow
import mlflow.pytorch
import abc

In [4]:
#Iniciando MlFlow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
#Definição do Environment em que o Agente atuará
class PortfolioEnv(gym.Env):
    def __init__(self,arquivo_dados):
        """
        Inicializa o ambiente.
        :param dados_preco: DataFrame ou numpy array com preços dos ativos.
                           Exemplo: colunas ['Gold', 'IBOVESPA', 'Bitcoin']
        :param dados_observacoes: DataFrame ou numpy array com observação do modelo 
                           Exemplo: IPCA.
        """
        self.arquivo_dados = arquivo_dados
        dados = pd.read_csv(f'../data/{self.arquivo_dados}')
        self.dados_preco = dados.iloc[:, 1:4]  # Dataframe com preços dos ativos
        self.dados_observacao = dados.iloc[:, 4]  # Dataframe com Observações (IPCA)
        
        #Começamos no dia 0
        self.dia_inicial = -1  
        self.dia = self.dia_inicial

        #Espaço de Ação e de Observação
        self.action_space = gym.spaces.Box(low=0.0, high=1.0, shape=(3,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=0.0, high=np.inf, shape=(5,), dtype=np.float32)

        # Estado inicial do portfólio
        self._valor_inicial_portfolio = 100  # Valor inicial do portfólio
         
         #Total de Dias de um Episódio
        self.dias_em_um_episodio = 30

        #Episodio começa como não terminado
        self._episode_ended = False
    
    def get_estado(self, data: int) -> np.array:
        '''
        Obtém o estado atual do ambiente.

        :param data: Dia atual.
        :return: Estado atual.
        '''
        estado = np.concatenate([
            # alocacao,
            # [self._valor_portfolio],
            # self.dados_preco.iloc[self.dia],
            [self.dados_observacao.iloc[data]]
        ])
        return estado
    
    @abc.abstractmethod
    def calcula_recompensa(alocacao: torch.Tensor) -> torch.Tensor:
        pass

    #Função Chamada no Enviroment ao Fim de um Episódio, retorna o estado que será usado pelo o agente
    def reset(self):
        #Iniciado Alocação do Portfolio Aleatóriamente
        estado_alocacao_aleatorio = np.random.uniform(0,1,3)
        estado_aleatorio_alocacao_normalizado = estado_alocacao_aleatorio/np.sum(estado_alocacao_aleatorio)

        #Passando para o dia Seguinte
        self.dia_inicial = self.dia_inicial + 1
        self.dia = self.dia_inicial

        #Iniciado valor do portfólio para o valor inicial
        self._valor_portfolio = self._valor_inicial_portfolio

        self._episode_ended = False
        
        #Criando vetor de estado concatenado -> Necessário para entrada na Rede Neural
        estado = self.get_estado(self.dia_inicial)
        return estado
    
    #Função Chamada dentro de cada episódio, Recebe parametros da rede neualpara gerar uma amostra de uma distribuição normal
    def step(self,alocacao):
        recompensa = self.calcula_recompensa(alocacao)

        #Passa para o próximo dia
        self.dia += 1

        estado = self.get_estado(self.dia)

        #Alteração dentro da função step do ambiente
        if self.dia - self.dia_inicial == 30:
            self._episode_ended = True
        return estado, recompensa  # Retorna o estado atual e a recompensa ao invés de chamar reset()

In [5]:
class AmbienteAgenteConservador(PortfolioEnv):
    '''
    Implementa um ambiente de classe filha que implementa a função
    calcula_recompensa para um agente conservador.
    '''
    def calcula_recompensa(self, alocacao: torch.Tensor) -> torch.Tensor:
        '''
        Calcula a recompensa para um agente conservador.

        Args:
            alocacao: A alocação de ativos.
        
        Returns:   
            A recompensa.
        '''
        #Calcula a valorização do portfolio
        valor_portfolio_pre_aplicacao = self._valor_portfolio
        preco_usado = self.dados_preco.iloc[self.dia].values
        preco_dia_seguinte = self.dados_preco.iloc[self.dia + 1].values
        variacao_percentual = (preco_dia_seguinte-preco_usado)/preco_usado
        valores_aportados = torch.multiply(alocacao,self._valor_portfolio)
        valores_aportados_ajustados = torch.multiply(valores_aportados, torch.tensor(1 + variacao_percentual))
        self._valor_portfolio = torch.sum(valores_aportados_ajustados)

        #Calculo de Recompensa
        recompensa = torch.subtract(self._valor_portfolio,valor_portfolio_pre_aplicacao)

        #Ajustando Recompensa para aumentar grandeza de perdas
        if recompensa < 0:
            recompensa = recompensa*2

          
        return recompensa

In [18]:
class AmbienteAgenteArrojado(PortfolioEnv):
    '''
    Implementa um ambiente de classe filha que implementa a função
    calcula_recompensa para um agente conservador.
    '''
    def calcula_recompensa(self, alocacao: torch.Tensor) -> torch.Tensor:
        '''
        Calcula a recompensa para um agente conservador.

        Args:
            alocacao: A alocação de ativos.
        
        Returns:   
            A recompensa.
        '''
        #Calcula a valorização do portfolio
        valor_portfolio_pre_aplicacao = self._valor_portfolio
        preco_usado = self.dados_preco.iloc[self.dia].values
        preco_dia_seguinte = self.dados_preco.iloc[self.dia + 1].values
        variacao_percentual = (preco_dia_seguinte-preco_usado)/preco_usado
        valores_aportados = torch.multiply(alocacao,self._valor_portfolio)
        valores_aportados_ajustados = torch.multiply(valores_aportados, torch.tensor(1 + variacao_percentual))
        self._valor_portfolio = torch.sum(valores_aportados_ajustados)

        #Calculo de Recompensa
        recompensa = torch.subtract(self._valor_portfolio,valor_portfolio_pre_aplicacao)
         
        if recompensa < 0 :
            recompensa = 0.8 * recompensa
        elif recompensa > 0:
            recompensa = recompensa * 1.2
        return recompensa

In [7]:
#Definição de Rede Neural que servirá como Política
class PolicyNetwork(nn.Module):
    '''
    Observações:
    1. Necessitamos de um modo de gerar desvio padrão positivo e diferente de 0 
    2. Adicionar Saídas
    '''
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 16)  # Camada oculta 1
        self.fc1_ativ = nn.ReLU()
        self.fc2 = nn.Linear(16, 16)         # Camada oculta 2
        self.fc2_ativ = nn.ReLU()
        self.fc_mu = nn.Linear(16, action_dim)  # Média (mu) dos pesos de portfólio
        self.output_ativ = nn.Sigmoid()
      #  self.fc_sigma = nn.Linear(16, action_dim)  # Desvio padrão (sigma) dos pesos
    
    def forward(self, x):
        x = self.fc1_ativ(self.fc1(x))  # Passa pela camada oculta 1
        x = self.fc2_ativ(self.fc2(x))      # Passa pela camada oculta 2
        mu = self.output_ativ(self.fc_mu(x))  # Média (mu), usando tanh para limitar a saída
       # sigma = nn.functional.softplus(self.fc_sigma(x)) + 1e-6 # Desvio padrão (sigma), softplus para garantir positividade
        return mu#, sigma

In [13]:
#Função para treinamento do Agente
def train(ambiente,politica,otimizador,num_episodios,fator_desconto_recompensa):
    with mlflow.start_run():
        resultados_treino = pd.DataFrame(columns=["Episodio","Dia","Estado","Media","Desvio","Alocacao","Log_Prob","Recompensa"])
        for episodio in range(num_episodios):
            #Logar Parametros no MLFLow
            mlflow.log_params({"Número de Episódios":num_episodios,
                               "Learning Rate":otimizador.defaults['lr'],
                               "Fator de Desconto":fator_desconto_recompensa,
                               "Arquivo Usado": ambiente.arquivo_dados})
            
            if isinstance(ambiente,AmbienteAgenteArrojado):
                mlflow.log_param("Perfil do Cliente","Arrojado")
            elif isinstance(ambiente,AmbienteAgenteConservador):
                mlflow.log_param("Perfil do Cliente","Conservador")
                
            estado = ambiente.reset() #No inicio do episodo pegamos o estado inicial do ambiente
            mlflow.log_param("Dimensão do Estado",len(estado))
            mlflow.log_param("Função de Ativação Output",politica.output_ativ)
            recompensas_do_episodio = [] #Array para guardar as recompensas de cada episodio
            log_probs = [] #Array para guardar as probabilidades logaritimicas usadas no calculo do Gradiente de Politica
            estados =[]
            alocacoes= []
            dias = []
            medias = []
            desvios = []
        
            while not ambiente._episode_ended:
                #Recebendo Estado
                estado_tensor = torch.tensor(estado, dtype=torch.float32) #Transformarmos em tensor -> Requisitado pelo Pytorch
            # media,desvio_padrao = politica(estado_tensor) #Geramos os parametros para criação da distribuição da ação
                media = politica(estado_tensor)
                medias.append(media)   
                desvio_padrao = torch.tensor([0.1,0.1,0.1])
                desvios.append(desvio_padrao)
            # desvios.append(desvio_padrao)

                #Adicionando dia
                dias.append(ambiente.dia)

                #Gerando Alocação
                # Função para amostrar os pesos do portfólio
                def sample_portfolio_weights(mu, sigma):
                    dist = torch.distributions.Normal(mu, sigma)  # Cria a distribuição normal
                    action = dist.rsample()  # Amostragem reparametrizada
                    log_prob = dist.log_prob(action).sum(dim=-1)  # Log-probabilidade da ação
                    action = action/torch.sum(action)
                    return action, log_prob
                
                # Amostra os pesos do portfólio
                alocacao, log_prob = sample_portfolio_weights(media,desvio_padrao)
                

                #Guardando Estado e Alocação
                estados.append(estado)
                alocacoes.append(alocacao)

                #Calculando log-probabilidade da ação
                log_probs.append(log_prob)

                #Executa a alocação e recebe a recompensa
                retorno = ambiente.step(alocacao)
                estado = retorno[0] #Atualiza o estado para o novo estado devolvido pelo ambiente e pega a recompensa da ação anterior
                recompensa = retorno[1]
                recompensas_do_episodio.append(recompensa)
            #Ao final do Episódio Calcular as recompensas com desconto
            #descontos = np.array([fator_desconto_recompensa**i for i in range(len(recompensas_do_episodio))])
            #recompensas_com_desconto = np.array(recompensas_do_episodio) * descontos

            #Calculo de Perda -> Gradiente de Politica:
            perda = -torch.sum(torch.stack(log_probs) * torch.stack(recompensas_do_episodio))
            otimizador.zero_grad()
            perda.backward()
            otimizador.step()
            
            # Adicionar resultados em um dataframe para análise do treinamento
            for dia, estado, media, desvio, alocacao, log_prob, recompensa in zip(dias, estados, medias, desvios, alocacoes, log_probs, recompensas_do_episodio):
                resultados_treino.loc[len(resultados_treino)]= {
                    "Episodio": episodio,
                    "Dia": dia,
                    "Estado": estado,
                    "Media": media.detach().numpy(),
                    "Desvio": desvio.detach().numpy(),
                    "Alocacao": alocacao.detach().numpy(),
                    "Log_Prob": log_prob.detach().numpy(),
                    "Recompensa": recompensa.detach().numpy(),
                }
        
        #Salvando Treino 
        def salvartreino():
            media_final = medias[-1].detach().numpy()
            for i, valor in enumerate(media_final):
                mlflow.log_metric(f"Média Final_{i}", valor)

            # Desvio Final
            desvio_final = desvios[-1].detach().numpy()
            for i, valor in enumerate(desvio_final):
                mlflow.log_metric(f"Desvio Final_{i}", valor)

            # Alocação Final
            alocacao_final = alocacoes[-1].detach().numpy()
            for i, valor in enumerate(alocacao_final):
                mlflow.log_metric(f"Alocação Final_{i}", valor)

            #Recompensa Média por Episódio
            mlflow.log_metric("Recompensa Média por Episódio", resultados_treino.groupby('Episodio')['Recompensa'].sum().mean())
            mlflow.log_metric("Dias por Episódio", ambiente.dias_em_um_episodio)
            max_n = 0
            for i in os.listdir('../data/resultados_treinos/v1.1'):
                n = i.split('_')[1]
                n = int(n.split('.')[0])
                if int(n) > max_n:
                    max_n = n 
            max_n = max_n +1
            resultados_treino.to_csv(f"../data/resultados_treinos/v1.1/treino_{max_n}.csv", index=False)
             # Logar o arquivo no MLflow
            mlflow.log_artifact(f"../data/resultados_treinos/v1.1/treino_{max_n}.csv")
            mlflow.pytorch.log_model(pytorch_model=politica,artifact_path='Magnum')
        salvartreino()
       
        return resultados_treino

In [14]:
#Treino Conservador
ambiente = AmbienteAgenteConservador('artificial_v2.csv') #Criando Ambiente
politica = PolicyNetwork(1, 3) #Criando Politica Estocástica
otimizador = optim.Adam(politica.parameters(),lr=0.01) #Cria otimizador associado aos parâmetros da rede a ser atualizada
resultados_treino = train(ambiente,politica,otimizador,num_episodios=1000,fator_desconto_recompensa=0.99)
resultados_treino.head()

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


,Episodio,Dia,Estado,Media,Desvio,Alocacao,Log_Prob,Recompensa
0,0,0,[0.38],"[0.41775882, 0.47877842, 0.46034038]","[0.1, 0.1, 0.1]","[0.31602177, 0.37927696, 0.30470124]",3.967723,0.006936799402467386
1,0,1,[0.38],"[0.41775882, 0.47877842, 0.46034038]","[0.1, 0.1, 0.1]","[0.2504626, 0.17158228, 0.5779551]",-3.346537,0.003729934905990717
2,0,2,[0.38],"[0.41775882, 0.47877842, 0.46034038]","[0.1, 0.1, 0.1]","[0.35997513, 0.22494039, 0.4150845]",0.5674087,0.004696119786345321
3,0,3,[0.38],"[0.41775882, 0.47877842, 0.46034038]","[0.1, 0.1, 0.1]","[0.3311068, 0.30012205, 0.36877114]",3.6930184,0.005747650059902298
4,0,4,[0.38],"[0.41775882, 0.47877842, 0.46034038]","[0.1, 0.1, 0.1]","[0.2858466, 0.3592052, 0.35494816]",3.0554,0.006517263752812141


In [19]:
#Treino Arrojado
ambiente = AmbienteAgenteArrojado('artificial_v2.csv') #Criando Ambiente
politica = PolicyNetwork(1, 3) #Criando Politica Estocástica
otimizador = optim.Adam(politica.parameters(),lr=0.1) #Cria otimizador associado aos parâmetros da rede a ser atualizada
resultados_treino = train(ambiente,politica,otimizador,num_episodios=1000,fator_desconto_recompensa=0.99)
resultados_treino.head()

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


,Episodio,Dia,Estado,Media,Desvio,Alocacao,Log_Prob,Recompensa
0,0,0,[0.38],"[0.4906445, 0.5246462, 0.4798585]","[0.1, 0.1, 0.1]","[0.25849342, 0.3595814, 0.38192523]",3.0464144,0.00787445799468287
1,0,1,[0.38],"[0.4906445, 0.5246462, 0.4798585]","[0.1, 0.1, 0.1]","[0.26066965, 0.37453708, 0.36479324]",3.1315813,0.00812328954136774
2,0,2,[0.38],"[0.4906445, 0.5246462, 0.4798585]","[0.1, 0.1, 0.1]","[0.33770424, 0.36364052, 0.29865527]",2.2567682,0.008068563349576152
3,0,3,[0.38],"[0.4906445, 0.5246462, 0.4798585]","[0.1, 0.1, 0.1]","[0.3659752, 0.3340899, 0.29993483]",3.8655562,0.007566955743385506
4,0,4,[0.38],"[0.4906445, 0.5246462, 0.4798585]","[0.1, 0.1, 0.1]","[0.2884731, 0.43184116, 0.2796857]",3.024898,0.009118035606718422
